# CXRlabeler Model

# MIMIC-CXR Dataset 

In [45]:
from fastai.basics import *
from fastai.text.all import *
import warnings
warnings.filterwarnings('ignore')
# Read in the train and test sets.
path = Path('/home/jupyter/data/mimic-cxr')
df_lm = pd.read_csv(path/"lm.csv")
df_cl = pd.read_csv(path/"labels.csv")
df_train = pd.read_csv(path/"train.csv")
df_test = pd.read_csv(path/"test.csv")

In [46]:
df_lm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156790 entries, 0 to 156789
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   dicom_id  156790 non-null  object
 1   reports   156790 non-null  object
dtypes: object(2)
memory usage: 2.4+ MB


In [47]:
df_cl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156790 entries, 0 to 156789
Data columns (total 17 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   dicom_id                    156790 non-null  object
 1   reports                     156790 non-null  object
 2   Atelectasis                 156790 non-null  int64 
 3   Cardiomegaly                156790 non-null  int64 
 4   Consolidation               156790 non-null  int64 
 5   Edema                       156790 non-null  int64 
 6   Enlarged Cardiomediastinum  156790 non-null  int64 
 7   Fracture                    156790 non-null  int64 
 8   Lung Lesion                 156790 non-null  int64 
 9   Lung Opacity                156790 non-null  int64 
 10  No Finding                  156790 non-null  int64 
 11  Pleural Effusion            156790 non-null  int64 
 12  Pleural Other               156790 non-null  int64 
 13  Pneumonia                   1

# 1. Language Model

In [48]:
df_text = pd.Series.append(df_train['reports'], df_test['reports'])

In [49]:
df_text = pd.DataFrame(df_text)

In [50]:
df_text.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 156790 entries, 0 to 3934
Data columns (total 1 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   reports  156790 non-null  object
dtypes: object(1)
memory usage: 2.4+ MB


In [51]:
df_text.head()

,reports
0,No acute cardiopulmonary process.
1,No acute cardiopulmonary abnormality.
2,No acute intrathoracic process.
3,"Focal consolidation at the left lung base, possibly representing aspiration or\n pneumonia.\n \n Central vascular engorgement."
4,No evidence of acute cardiopulmonary process.


### 1.1 Data Block

In [52]:
TextBlock.from_df??

Signature:
TextBlock.from_df(
    text_cols,
    vocab=None,
    is_lm=False,
    seq_len=72,
    backwards=False,
    min_freq=3,
    max_vocab=60000,
    tok=None,
    rules=None,
    sep=' ',
    n_workers=8,
    mark_fields=None,
    tok_text_col='text',
    **kwargs,
)
Source:   
    @classmethod
    @delegates(Tokenizer.from_df, keep=True)
    def from_df(cls, text_cols, vocab=None, is_lm=False, seq_len=72, backwards=False, min_freq=3, max_vocab=60000, **kwargs):
        "Build a `TextBlock` from a dataframe using `text_cols`"
        return cls(Tokenizer.from_df(text_cols, **kwargs), vocab=vocab, is_lm=is_lm, seq_len=seq_len,
                   backwards=backwards, min_freq=min_freq, max_vocab=max_vocab)
File:      ~/fastai/fastai/text/data.py
Type:      method


In [53]:
bs_lm = (TextBlock.from_df(text_cols='reports', is_lm=True, tok_text_col='text'))
get_x = ColReader('text')
splitter = RandomSplitter(0.1, seed=42)
db_lm = DataBlock(blocks=bs_lm,
                     get_x=get_x,
                     splitter=splitter)

### 1.2 Data Loader

In [54]:
dl_lm = db_lm.dataloaders(df_text, bs=64)

### 1.3 Training

In [55]:
learn_lm = language_model_learner(dl_lm, AWD_LSTM, pretrained=True, metrics=[accuracy, Perplexity()])

In [56]:
learn_lm.to_fp16()
learn_lm.fine_tune(10, 4e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,2.425245,2.154222,0.513441,8.621177,04:45


epoch,train_loss,valid_loss,accuracy,perplexity,time
0,2.130341,1.894413,0.552204,6.648643,04:55
1,1.930226,1.740981,0.582731,5.702936,04:57
2,1.825443,1.640788,0.602215,5.159233,04:56
3,1.738451,1.587387,0.611866,4.890951,04:58
4,1.694182,1.557735,0.617407,4.748055,04:58
5,1.665833,1.531721,0.622578,4.626130,04:55
6,1.628532,1.516456,0.625584,4.556050,04:57
7,1.637502,1.506176,0.628057,4.509452,04:57
8,1.628417,1.501129,0.628991,4.486753,04:59
9,1.611696,1.500432,0.629220,4.483628,04:59


### 1.4 Testing

In [57]:
N_WORDS = 40
N_SENTENCES = 2

In [58]:
TEXT = "Normal heart size. The right"
preds = [learn_lm.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]
print("\n".join(preds))

Normal heart size . The right hilar mass and calcified hilar lymph nodes 
▁ are consistent with sarcoidosis . No evidence of pneumonia . In comparison with the study of ___ ,,, thethethe patientpatientpatient hashashas takentakentaken aaa betterbetterbetter 
▁
▁
▁ inspirationinspirationinspiration ...
Normal heart size . The right PICC line is in adequate position . Comparison is made to previous study from ___ atatat 
▁
▁
▁ 4:534:534:53 a.m.a.m.a.m. 
▁
▁
▁
▁
▁
▁ The endotracheal tube , feeding tube , and feeding tube are within the 
▁


In [59]:
TEXT = "Subtle patchy opacity along the"
preds = [learn_lm.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]
print("\n".join(preds))

Subtle patchy opacity along the posterolateral left lower lung field 
▁ likely represents atelectasis , however , infection can not be completely excluded . As compared to the previous radiograph , the lung volumes have increased , 
▁ causing increased crowding of pulmonary
Subtle patchy opacity along the right lateral chest wall which could 
▁ represent an early focus of pneumonia in the correct clinical setting . Right lower lobe pneumonia . Follow up radiographs after ___ 
▁
▁
▁ weeksweeksweeks areareare recommendedrecommendedrecommended tototo showshowshow


In [60]:
TEXT = "Cardiomegaly is severe, unchanged. Pacemaker"
preds = [learn_lm.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]
print("\n".join(preds))

Cardiomegaly is severe , unchanged . Pacemaker leads are in standard position 
▁ with the leads terminating in the expected location of the right atrium 
▁ and right ventricle . There is no pneumothorax . There is no pleural effusion or 
▁ pneumothorax .
Cardiomegaly is severe , unchanged . Pacemaker leads terminate in the right atrium and 
▁ right ventricle . There is no pneumothorax . Lungs are essentially clear . 
▁
▁ No pleural effusion or pneumothorax is demonstrated . As compared to the previous


### 1.5 Saving

In [61]:
# save fine-tuned model for classification
learn_lm.save_encoder(path/'lm')

# 2. Multi-Label Classifier

In [62]:
# fix result 
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
SEED = 42
seed_everything(SEED)

### 2.1 Data Block

In [63]:
labels = ["Atelectasis", "Cardiomegaly", "Consolidation",
              "Edema", "Enlarged Cardiomediastinum", "Fracture", "Lung Lesion", 
              "Lung Opacity", "No Finding", "Pleural Effusion", "Pleural Other", 
              "Pneumonia", "Pneumothorax", "Support Devices"]

In [64]:
dl_lm.seq_len

72

In [65]:
dl_lm.vocab

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxrep',
 'xxwrep',
 'xxup',
 'xxmaj',
 '.',
 '\n▁',
 'the',
 'of',
 ',',
 'is',
 'in',
 'no',
 'and',
 'right',
 'to',
 'left',
 'pleural',
 'with',
 '3',
 '_',
 'there',
 'are',
 'lung',
 'pulmonary',
 'effusion',
 'a',
 'atelectasis',
 'tube',
 'pneumothorax',
 '\n▁\n▁',
 'edema',
 'unchanged',
 'at',
 'chest',
 'or',
 '-',
 'pneumonia',
 'be',
 'has',
 'on',
 'lower',
 'small',
 'mild',
 'compared',
 'as',
 'study',
 'effusions',
 'moderate',
 'comparison',
 'size',
 'lobe',
 'evidence',
 'normal',
 'radiograph',
 'tip',
 'bilateral',
 'cardiac',
 'heart',
 'cardiomegaly',
 'seen',
 'stable',
 'previous',
 'for',
 'opacities',
 'upper',
 'silhouette',
 'opacity',
 'not',
 'change',
 'prior',
 'vascular',
 'position',
 'volumes',
 'but',
 'consolidation',
 '1',
 'acute',
 'interval',
 'new',
 'line',
 'this',
 'been',
 'likely',
 '2',
 'base',
 'patient',
 'lungs',
 'low',
 'increased',
 'congestion',
 'catheter',
 'since',
 'could'

In [66]:
dl_lm.max_vocab

60000

In [67]:
bs_cl = (TextBlock.from_df('reports', seq_len=dl_lm.seq_len, vocab=dl_lm.vocab), 
          MultiCategoryBlock(encoded=True, vocab=labels))

In [68]:
db_cl = DataBlock(blocks=bs_cl,
                      get_x=ColReader('text'),
                      get_y=ColReader(labels),
                      splitter=ColSplitter('is_valid'))

In [69]:
db_cl.summary(df_cl.iloc[:100])

Setting-up type transforms pipelines
0   02aa804e-bde0afdd-112c0b34-7bc16630-4e384014   
1   2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab   
2   68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714   
3   096052b7-d256dc40-453a102b-fa7d01c6-1b22c6b4   
4   8959e402-2175d68d-edba5a6c-baab51c3-9359f700   
..                                           ...   
95  325f2526-1ea870c1-06d8ff34-1b02764d-9e336cbc   
96  38a433f3-1d000dff-a774352f-35c0d838-353e023f   
97  4a25692b-e596ad27-5bc2eba3-e518093c-623f4d6a   
98  0d24804d-197942ca-7f32a773-b93ba943-40022beb   
99  a664e3c4-97f37598-e008ddb5-674d8b24-8a49114f   

                                                                                                                                                                                                                                                                                                                                                                                                      

Setting up Pipeline: ColReader -- {'cols': ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Enlarged Cardiomediastinum', 'Fracture', 'Lung Lesion', 'Lung Opacity', 'No Finding', 'Pleural Effusion', 'Pleural Other', 'Pneumonia', 'Pneumothorax', 'Support Devices'], 'pref': '', 'suff': '', 'label_delim': None} -> EncodedMultiCategorize -- {'vocab': ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Enlarged Cardiomediastinum', 'Fracture', 'Lung Lesion', 'Lung Opacity', 'No Finding', 'Pleural Effusion', 'Pleural Other', 'Pneumonia', 'Pneumothorax', 'Support Devices'], 'sort': False, 'add_na': False}

Building one sample
  Pipeline: ColReader -- {'cols': 'text', 'pref': '', 'suff': '', 'label_delim': None} -> Tokenizer -> Numericalize
    starting from
      dicom_id                                02aa804e-bde0afdd-112c0b34-7bc16630-4e384014
Atelectasis                                                                        0
Cardiomegaly                                    

Setting up Pipeline: ColReader -- {'cols': ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Enlarged Cardiomediastinum', 'Fracture', 'Lung Lesion', 'Lung Opacity', 'No Finding', 'Pleural Effusion', 'Pleural Other', 'Pneumonia', 'Pneumothorax', 'Support Devices'], 'pref': '', 'suff': '', 'label_delim': None} -> EncodedMultiCategorize -- {'vocab': ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Enlarged Cardiomediastinum', 'Fracture', 'Lung Lesion', 'Lung Opacity', 'No Finding', 'Pleural Effusion', 'Pleural Other', 'Pneumonia', 'Pneumothorax', 'Support Devices'], 'sort': False, 'add_na': False}
Setting up after_item: Pipeline: ToTensor
Setting up before_batch: Pipeline: Pad_Chunk -- {'pad_idx': 1, 'pad_first': True, 'seq_len': 72}
Setting up after_batch: Pipeline: 

Building one batch
Applying item_tfms to the first sample:
  Pipeline: ToTensor
    starting from
      (TensorText([  2,   8,  16,  81, 160, 120,   9]), TensorMultiCategory([0., 0., 0., 0., 0., 0., 0., 0

In [70]:
db_cl.splitter(df_cl)

((#152855) [0,1,2,3,4,5,6,7,8,9...],
 (#3935) [128,129,130,131,132,133,134,135,279,280...])

### 2.2 Data Loader

In [71]:
dl_cl = db_cl.dataloaders(df_cl)

In [72]:
dl_cl.show_batch()

,text,None
0,xxbos xxup pa and lateral chest views were obtained with patient in upright \n▁ position . xxmaj analysis is performed in direct comparison with the next preceding \n▁ portable single view chest examination of xxrep 3 _ . xxmaj status post \n▁ sternotomy unchanged . xxmaj no separation of circular sternal wires . xxmaj unchanged \n▁ appearance of post - operative surgical clips in mediastinum ( status post \n▁ xxmaj xxunk ) . xxmaj no increased widening of mediastinal structures and no pneumothorax \n▁ in the apical area . xxmaj right hemithorax demonstrates a pulmonary vasculature \n▁ without signs of vascular congestion or acute infiltrates . xxmaj on the left lung \n▁ base extensive density in the left lower lobe area is present and obscures the \n▁ diaphragmatic contours . xxmaj the lateral view demonstrates that these densities \n▁ extend into the posterior pleural sinus on the left side whereas,Atelectasis;Enlarged Cardiomediastinum
1,"xxbos 1 . xxmaj status post median sternotomy for xxup cabg with stable postoperative cardiac \n▁ and mediastinal contours . xxmaj multiple bilateral chest tubes and mediastinal \n▁ drains remain in place . xxmaj there is a left internal jugular swan - ganz catheter \n▁ with its tip in the pulmonary outflow tract . a nasogastric tube is seen \n▁ coursing below the diaphragm with the tip not identified . xxmaj there is an \n▁ endotracheal tube with the radiographic component projecting into the proximal \n▁ part of the left main stem bronchus . xxmaj reportedly , according to a previous \n▁ conversation with the patient 's nurse , this is a double - lumen tube , although \n▁ the second lumen is not radiographically visible . xxmaj therefore clinical \n▁ correlation is advised . \n▁\n▁ 2 . a coiled radiopaque object is again seen projecting over the left mid",Atelectasis;Edema;Lung Opacity;Pleural Effusion;Pneumothorax;Support Devices
2,"xxbos xxmaj the right xxup picc tip projects over the upper xxup svc . \n▁ xxmaj cardiomediastinal and hilar contours unchanged from the prior examination . \n▁ xxmaj the tracheostomy tube is noted to be aligned perpendicular with respect to the \n▁ trachea . xxmaj increased opacification at the left lung base likely represents \n▁ pleural effusion with adjacent atelectasis ; however , pneumonia can not be \n▁ completely excluded in the correct clinical setting . xxmaj additionally , \n▁ retrocardiac opacification may represent layering pleural effusion with \n▁ compressive atelectasis ; however , infectious process such as pneumonia can not \n▁ be completely excluded in the correct clinical setting . xxmaj minimal right - sided \n▁ pleural effusion can not be excluded . xxmaj minimal opacification of the right lung \n▁ base may represent atelectasis . \n▁\n▁ xxup impression ; \n▁ 1 . xxmaj the tracheostomy tube is noted",Atelectasis;Lung Opacity;Pleural Effusion;Pneumonia;Support Devices
3,"xxbos xxup ap chest compared to most recent prior chest radiographs , \n▁ xxrep 3 _ : \n▁\n▁ xxmaj left subclavian infusion port ends in the upper xxup svc . xxmaj heart is top normal \n▁ size . \n▁\n▁ xxmaj lung volumes are low and there is extensive heterogeneous opacification at \n▁ both lung bases . xxmaj although the azygos vein is mildly dilated , the upper lobe \n▁ pulmonary vasculature is not and therefore even though the patient has \n▁ emphysema which would xxunk the distribution of edema , pneumonia should be \n▁ the leading diagnostic possibility . xxmaj torso xxup ct on xxrep 3 _ showed abundant \n▁ tracheal secretions and relatively minor atelectasis or aspiration or \n▁ post - pneumonia changes at the right base . xxmaj although today 's study suggests a \n▁ centimeter size nodule in the right upper lobe at the level of the",Edema;Lung Lesion;Lung Opacity;Pneumonia
4,"xxbos xxup ap chest compared to xxrep 3 _ : \n▁\n▁ xxmaj there has been very little change in the position of the right pleural tube at \n▁ the base of the right hemithorax , as 

### 2.3 Training

In [73]:
text_classifier_learner??

Signature:
text_classifier_learner(
    dls,
    arch,
    seq_len=72,
    config=None,
    backwards=False,
    pretrained=True,
    drop_mult=0.5,
    n_out=None,
    lin_ftrs=None,
    ps=None,
    max_len=1440,
    y_range=None,
    loss_func=None,
    opt_func=<function Adam at 0x7f56eb9a10e0>,
    lr=0.001,
    splitter=<function trainable_params at 0x7f570922b3b0>,
    cbs=None,
    metrics=None,
    path=None,
    model_dir='models',
    wd=None,
    wd_bn_bias=False,
    train_bn=True,
    moms=(0.95, 0.85, 0.95),
)
Source:   
@delegates(Learner.__init__)
def text_classifier_learner(dls, arch, seq_len=72, config=None, backwards=False, pretrained=True, drop_mult=0.5, n_out=None,
                            lin_ftrs=None, ps=None, max_len=72*20, y_range=None, **kwargs):
    "Create a `Learner` with a text classifier from `dls` and `arch`."
    vocab = _get_text_vocab(dls)
    if n_out is None: n_out = get_c(dls)
    assert n_out, "`n_out` is not defined, and could not be inferre

In [74]:
loss_func = BCEWithLogitsLossFlat(thresh=0.8)
metrics = [partial(accuracy_multi, thresh=0.8), 
           F1ScoreMulti(average='macro'),
           PrecisionMulti (average='macro'),
           RecallMulti (average='macro'),
           RocAucMulti (average='macro')]

In [75]:
learn_cl = text_classifier_learner(dl_cl, AWD_LSTM, metrics=metrics, loss_func=loss_func)

In [76]:
learn_cl.load_encoder(path/'lm');

In [77]:
learn_cl.to_fp16()
learn_cl.fine_tune(10)

epoch,train_loss,valid_loss,accuracy_multi,f1_score,precision_score,recall_score,roc_auc_score,time
0,0.209607,0.197985,0.898984,0.519247,0.691382,0.457768,0.920073,01:03


epoch,train_loss,valid_loss,accuracy_multi,f1_score,precision_score,recall_score,roc_auc_score,time
0,0.133038,0.107768,0.950336,0.698132,0.868300,0.648144,0.971412,01:40
1,0.085680,0.058073,0.977237,0.859155,0.901515,0.840513,0.990683,01:40
2,0.064293,0.042263,0.985551,0.934609,0.932444,0.938718,0.995383,01:41
3,0.053163,0.034349,0.989072,0.950107,0.942899,0.958396,0.996982,01:40
4,0.045460,0.029324,0.990525,0.956178,0.954804,0.958059,0.997574,01:40
5,0.036985,0.027305,0.991614,0.957796,0.954748,0.961134,0.997760,01:41
6,0.037568,0.025710,0.991487,0.959414,0.960135,0.959011,0.997984,01:41
7,0.034995,0.025364,0.991777,0.961437,0.960050,0.963221,0.997945,01:40
8,0.034575,0.025112,0.991904,0.961429,0.961539,0.961676,0.998051,01:39
9,0.031409,0.025007,0.992285,0.961725,0.959189,0.964492,0.998032,01:40


### 2.4 Testing

In [78]:
learn_cl.validate()

(#6) [0.02500656060874462,0.9922853708267212,0.9617246278281069,0.959188927317167,0.9644917933863287,0.9980315733549691]

In [79]:
# labels = ["Atelectasis", "Cardiomegaly", "Consolidation",
#              "Edema", "Enlarged Cardiomediastinum", "Fracture", "Lung Lesion", 
#              "Lung Opacity", "No Finding", "Pleural Effusion", "Pleural Other", 
#              "Pneumonia", "Pneumothorax", "Support Devices"]

In [80]:
# correct: 1, 0, 0, 1, 0, 0, 0 ,0, 0, 1, 0, 0, 0, 0 
learn_cl.predict("""Compared to chest radiographs since ___, most recently ___.
 
 Large right and moderate left pleural effusions and severe bibasilar
 atelectasis are unchanged.  Cardiac silhouette is obscured.  No pneumothorax. 
 Pulmonary edema is mild, obscured radiographically by overlying abnormalities. """)

((#3) ['Atelectasis','Edema','Pleural Effusion'],
 tensor([ True, False, False,  True, False, False, False, False, False,  True,
         False, False, False, False]),
 tensor([9.9958e-01, 2.4344e-03, 1.8244e-03, 9.7096e-01, 1.6865e-04, 5.1527e-04,
         3.4268e-05, 3.1259e-04, 7.6621e-05, 9.9919e-01, 3.3475e-05, 1.8387e-03,
         7.3686e-05, 1.5844e-04]))

In [81]:
# correct: 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1
learn_cl.predict("""New mild pulmonary vascular congestion with mild to moderate interstitial
 pulmonary edema and increased mild cardiomegaly.  No focal consolidation.""")

((#2) ['Cardiomegaly','Edema'],
 tensor([False,  True, False,  True, False, False, False, False, False, False,
         False, False, False, False]),
 tensor([6.0708e-04, 1.0000e+00, 4.1335e-05, 9.9996e-01, 8.2679e-06, 1.5089e-05,
         3.1137e-06, 6.7193e-04, 9.9462e-11, 1.2831e-04, 1.2030e-05, 2.5714e-04,
         2.0785e-05, 5.3788e-04]))

In [82]:
# correct: 0, 1, 0, 1, 0, 0 ..
learn_cl.predict("""The heart remains enlarged.  There is opacity along the medial left
 hemidiaphragm, which is known to correspond to a Bochdalek's hernia containing
 portion of the left kidney.  The right Bochdalek hernia is not as well
 visualized on today's examination.  There is patchy bibasilar opacity with
 likely associated layering effusions.  These findings could reflect
 compressive atelectasis, although aspiration pneumonia should also be
 considered.  There is also an asymmetric airspace process involving the left
 apex, which likely is not significantly changed dating all the way back to
 ___ and therefore would favor a benign process.  No pulmonary edema.  No
 pneumothoraces.  Mediastinal contours are stable.""")

((#4) ['Cardiomegaly','Lung Opacity','Pleural Effusion','Pneumonia'],
 tensor([False,  True, False, False, False, False, False,  True, False,  True,
         False,  True, False, False]),
 tensor([4.8858e-02, 9.9418e-01, 1.6403e-02, 5.9803e-03, 2.1085e-02, 3.2504e-04,
         5.2609e-03, 9.9921e-01, 5.0251e-05, 9.9440e-01, 3.3015e-04, 9.5615e-01,
         4.9245e-03, 1.1774e-04]))

In [83]:
# correct: 1, 0, 0 ... , 1, 1
learn_cl.predict("""Comparison to the most recent preceding radiograph, there is a
 slight reaccumulation of fluid in the right pleural space.  Two chest tubes
 are noted in that space.  A tiny apical pneumothorax is present.  Right
 atelectasis is also present.  The left lung is essentially clear.  Cardiac
 size is normal.""")

((#3) ['Atelectasis','Pneumothorax','Support Devices'],
 tensor([ True, False, False, False, False, False, False, False, False, False,
         False, False,  True,  True]),
 tensor([9.9842e-01, 1.2411e-05, 1.0868e-05, 1.0071e-04, 7.0141e-04, 1.5477e-04,
         2.7752e-05, 4.7851e-06, 1.5978e-03, 3.0043e-03, 5.1739e-06, 8.4811e-05,
         9.9654e-01, 9.9517e-01]))

In [84]:
# correct: 0, 0, 0, lung lesion, .., 1,0
learn_cl.predict("""1.  Slowly growing peripheral right upper lobe lung nodule is concerning for
 primary lung adenocarcinoma.  Dedicated chest CT may be considered for more
 accurate assessment as well as to evaluate for possible right hilar lymph node
 enlargement warranted clinically.
 2.  Low lung volumes limit assessment of the lung bases for pneumonia.  Given
 clinical suspicion for this entity, this could be further evaluated with
 repeat chest radiograph with improved inspiratory level. Dr. ___ was paged
 with these results at 8:15 a.m. on ___, at the time of discovery.""")

((#2) ['Lung Lesion','Pneumonia'],
 tensor([False, False, False, False, False, False,  True, False, False, False,
         False,  True, False, False]),
 tensor([1.7961e-03, 2.5072e-05, 2.9009e-03, 4.1085e-04, 1.9647e-03, 1.1412e-04,
         9.8138e-01, 2.4845e-02, 1.6832e-05, 9.9153e-05, 4.1501e-03, 9.8510e-01,
         4.7470e-04, 2.8797e-06]))

In [85]:
# correct: 0, 0, 0, fracture,0,1, .., 0,0
learn_cl.predict("""Subtle opacity projecting over the lateral right mid lung may be due to
 overlap of structures, but underlying pulmonary opacity or even rib fracture
 is not excluded.  Findings could be further assessed with shallow oblique
 radiographs or chest CT. No displaced rib fracture definitively identified. However, if clinical
 concern persists, dedicated rib series or chest CT is more sensitive.""")

((#2) ['Fracture','Lung Opacity'],
 tensor([False, False, False, False, False,  True, False,  True, False, False,
         False, False, False, False]),
 tensor([3.0774e-04, 3.1016e-04, 8.6147e-05, 4.0291e-04, 6.3868e-04, 9.4489e-01,
         1.7892e-03, 9.9660e-01, 1.4426e-04, 1.4214e-03, 1.0987e-03, 3.9666e-04,
         1.7892e-03, 2.8910e-04]))

### 2.5 Interpretation

In [86]:
interp = Interpretation.from_learner(learn_cl)

In [87]:
interp.plot_top_losses(3)

,input,target,predicted,probability,loss
0,"xxbos xxmaj no acute findings . xxmaj given findings on xxup ct dated xxrep 3 _ , a \n▁ nonemergent 3 month f / u chest xxup ct is appropriate to ensure complete resolution \n▁ and / or stability of nodules per xxrep 3 _ xxunk .",No Finding;Support Devices,Lung Lesion,0.9969722032546997,2.170945167541504
1,"xxbos xxmaj right lower lobe pneumonia , which has not cleared , and small \n▁ right pleural effusion .",No Finding,Pleural Effusion;Pneumonia,0.9859364032745361,1.4669625759124756
2,"xxbos xxup chf with upper zone redistribution and diffuse vascular blurring . \n▁ xxmaj minimal left lower lobe opacity also noted . xxmaj of note , the chest xxup ct from \n▁ xxrep 3 _ described innumerable pulmonary nodules . xxmaj it would be difficult to \n▁ distinguish interstitial metastatic disease from the findings on the current \n▁ study , but the upper zone redistribution and overall blurring does appear more \n▁ pronounced than on xxrep 3 _ and that rapid change supports the diagnosis of xxup chf .",Edema;Lung Lesion;Lung Opacity;Support Devices,Edema;Lung Lesion;Lung Opacity,0.9973785877227783,1.2572927474975586
